<b>In the previous chapter, we saw that cleaning effects of C from B helped us segregate B and C quite well</b>

As a simple sanity check, lets try segregating B from C on the dirty data

In [2]:
import pandas as pd
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt

C:\Users\sabri\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\sabri\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\sabri\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
B = pd.read_csv("../assets/data/data_B.csv", header=None)
C = pd.read_csv("../assets/data/data_C.csv", header=None)

In [4]:
B.shape, C.shape

((2639, 24), (2322, 24))

In [5]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix

In [10]:
model_classif_ABC = tf.keras.Sequential()
model_classif_ABC.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True), input_shape=(24, 1)))
model_classif_ABC.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='relu')))
model_classif_ABC.add(tf.keras.layers.Flatten())
model_classif_ABC.add(tf.keras.layers.Dropout(0.2))
model_classif_ABC.add(tf.keras.layers.Dense(12, activation='relu'))
model_classif_ABC.add(tf.keras.layers.Dense(2, activation='sigmoid'))
model_classif_ABC.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
def stat_summary(X, iqrl=0.25, iqrr=0.75):
    '''
    Return for each timestamp (hr) return min, max, 0.25 quantile, 0.75 quantile, mean, var
    '''
    return pd.DataFrame(np.array([np.min(X, axis=0), np.quantile(X, axis=0, q=iqrl), np.median(X, axis=0), 
          np.quantile(X, axis=0, q=iqrr), np.max(X, axis=0), np.mean(X, axis=0), np.var(X, axis=0), np.std(X, axis=0)]).transpose(),
                        columns=["Min", "0.25Q", "Med", "0.75Q", "Max", "Mean", "Var", "StD"])

def trend(X, span = None):
    '''
    Return EWMA trend for signal samples
    '''
    if not span:
        span = X.shape[1]
    return pd.DataFrame(X).apply(lambda x: x.ewm(span=span).mean(), axis=1)

def train_test_val_split(X, y=None, test_size=0.2,  val_size=0.4):
    from sklearn.model_selection import train_test_split
    if y.__len__() > 0:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
        X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=val_size)
        return X_train, X_test, X_val, y_train, y_test, y_val
    else:
        X_train, X_test = train_test_split(X, test_size=test_size)
        X_test, X_val = train_test_split(X_test, test_size=val_size)
        return X_train, X_test, X_val

def reshapeInput(X):
    X_ = []
    for x in X:
        X_.append(x.reshape(-1,1))
    return np.array(X_)

def shapeInput(X):
    X_ = []
    for x in X:
        X_.append(x.reshape(1,-1)[0])
    return np.array(X_)


def meanShift(X, window=None):
    '''
    Apply MA on convolved signal and compute rs means
    '''
    # At this point X must be EWMA convolved already
    if not window:
        window = X.shape[1] // 2
    return pd.DataFrame(X).apply(lambda x: x.rolling(window=window).mean(), axis=1).iloc[:,window-1:]

def gradient(X, shift=1, threshold=0):
    '''
    First order gradient of signal. Eq to diff for discrete signal
    '''
    return pd.DataFrame(X).apply(lambda x: x.diff(periods=shift), axis=1).iloc[:,threshold:]

def bucketPeaks(peakIndex):
    '''
    Bucket into first half and second half
    '''
    return(sum(peakIndex <= 12), sum(peakIndex > 12)) 

In [7]:
X = np.row_stack((B,C))
Y = [0]*(B.__len__()) + [1]*C.__len__()

In [8]:
X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X, tf.keras.utils.to_categorical(Y), test_size=0.1)

In [11]:
model_classif_ABC.fit(reshapeInput(X_train), y_train, 
          epochs=120, batch_size=100,
          validation_data=(reshapeInput(X_val), y_val),
         callbacks=tf.keras.callbacks.EarlyStopping(patience=25, restore_best_weights=True))

Epoch 1/120
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
40/45 [=========================>....] - ETA: 0s - loss: 0.6925 - accuracy: 0.5250WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x000001FB0ADE95E8> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`)

Epoch 45/120
45/45 [==============================] - 0s 11ms/step - loss: 0.6375 - accuracy: 0.6095 - val_loss: 0.6287 - val_accuracy: 0.6131
Epoch 46/120
45/45 [==============================] - 0s 11ms/step - loss: 0.6314 - accuracy: 0.6346 - val_loss: 0.6181 - val_accuracy: 0.6332
Epoch 47/120
45/45 [==============================] - 0s 11ms/step - loss: 0.6213 - accuracy: 0.6416 - val_loss: 0.6240 - val_accuracy: 0.5930
Epoch 48/120
45/45 [==============================] - 1s 11ms/step - loss: 0.6223 - accuracy: 0.6171 - val_loss: 0.6121 - val_accuracy: 0.6332
Epoch 49/120
45/45 [==============================] - 0s 11ms/step - loss: 0.6197 - accuracy: 0.6207 - val_loss: 0.6205 - val_accuracy: 0.6030
Epoch 50/120
45/45 [==============================] - 1s 11ms/step - loss: 0.6239 - accuracy: 0.6347 - val_loss: 0.6179 - val_accuracy: 0.6281
Epoch 51/120
45/45 [==============================] - 0s 11ms/step - loss: 0.6221 - accuracy: 0.6177 - val_loss: 0.6172 - val_accuracy: 0.6382

In [14]:
confusion_matrix(np.argmax(y_test, axis=1), model_classif_ABC.predict_classes(reshapeInput(X_test)))

C:\Users\sabri\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[111,  47],
       [ 60,  80]], dtype=int64)

While many B's were classified as B's, a lot of C's were misclassified as B's. This effect is due to signal pollution from tool C patterns, that we cleaned

<b>Also a sanity check would be to see if a simple ML model can perform just as well as the LSTM model we used in the previous chapter, ON THE CLEANED B vs C data<br/>
Lets use a ML model</b>

In [21]:
B = pd.read_csv("../assets/data/cleaned/B.csv", index_col=0)
C = pd.read_csv("../assets/data/cleaned/C.csv", index_col=0)

In [22]:
X = np.row_stack((B,C))
Y = [0]*(B.__len__()) + [1]*C.__len__()
X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X, tf.keras.utils.to_categorical(Y), test_size=0.1)

In [23]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier().fit(X_train, np.argmax(y_train, axis=1))
confusion_matrix(np.argmax(y_test, axis=1), rf.predict(X_test))

array([[44, 21],
       [16, 66]], dtype=int64)

Noice !!!